# DCF Part 1: WACC

In this notebook, we will compute the weighted average cost of capital.

$WACC = w_E * k_E + w_D * k_D * (1-t)$

Steps to get there:
1. Get firm data to compute Equity and Debt weights ($w_E$ and $w_D$)
2. Compute the cost of equity using the CAPM: $k_E = r_f + B_i * EMRP$

We'll get the risk free rate from market data.
We'll use an OLS regression to compute our own Beta.
The EMRP we will take as given.

3. Compute the cost of debt: $k_D = r_f + spread$

The $spread$ is the default spread given a firm's credit rating. We'll use a lookup table using current data to get the spread.

4. $t$ is the marginal tax rate, which we'll take as 25%.
5. Put it all together!

This is our in-class project that we will work on progressively through the mod.

Expectations:
1. Notebook is clean and neat, with no repeated code. It has clearly labeled sections for inputs/imports at the beginning. Code is sufficiently commented to demonstrate your understanding of the code and help you or anyone else who may use this code later. All numbers should be formatted so they are readable (so, use commas with large numbers, only a few decimal points).
2. All calculations are correct and all discussion questions are answered completely but concisely, demonstrating a depth of understanding.

Workflow:
1. Notebooks are inherently experimental and allow you to try things, however that requires some good habits
2. Once you are "done" in any sense, you always need to "clean up" your notebook to make it presentable. You'd do the same in Excel - you've tried lots of things, etc. but before you present it, you clean it up.
3. Finally, restart the Runtime/Kernel and run it cleanly all the way top to bottom one time.

Then, its ready to go.


# 1 Always put Imports and Installs at the beginning

And only put them in once.

Set display options here if you are using them.

Set API Keys here if you need them.

In [1]:
# install the FRED API for economic data
# This is necessary since Google Colab does not have this package installed already
#%%capture
!pip install fredapi

In [2]:
# necessary imports
import pandas as pd
import numpy as np
import yfinance as yf
import statsmodels.api as sm
from datetime import datetime
from fredapi import fred
from google.colab import userdata

In [3]:
# Format setting for the model - tame the decimals!
pd.options.display.float_format = '{:,.2f}'.format

In [8]:
# import
import fredapi
# These two lines set the API Key so you can access FRED data with your credentials
from google.colab import userdata
fred = fredapi.Fred(api_key=userdata.get('FRED_APIKey'))

# 2 Gather inputs, assumptions, parameters at the beginning

This is a place for anything that you may want to change in the future. The rest of the notebook should just be code that runs, generating the necessary output, plots, etc.

### Assumptions and paramters


In [11]:
# Set Ticker we are modeling
ticker_symbol = 'MSFT'
company_name = "Microsoft"

# choose the "market" for our beta - get from Yahoo! Finance
index_symbol = '^GSPC'
index_name = "S&P 500"

# get risk free rate from FRED
rf_rate = 'GS10'

# Initialize the Fred object
fred = fredapi.Fred(api_key='fc171a674a26eccd0c7493226c136a2d')

# first, get the whole series
rf_series = fred.get_series(rf_rate)

# get the most recent value, divide by 100 to make it a decimal like all others
risk_free_rate = float(rf_series.iloc[-1])  # convert to decimal immediately

# get the most recent yield
print(f"Latest 10-Year Treasury Yield: {risk_free_rate:.2f}% (as of {rf_series.index[-1].date()})")

emrp = 0.05 # given
marg_tax_rate = 0.25 # for WACC, given

# this is what we divide numbers by, 1000000 is $M
scale_factor = 1000000
scale_name = '$M'

Latest 10-Year Treasury Yield: 4.06% (as of 2025-10-01)


# 3 Begin Calculations and Modeling

**From here, nothing should be hardcoded**

## Debt and Equity Weights

In [12]:
# Create a Ticker object
ticker = yf.Ticker(ticker_symbol)

# Get the market cap
market_cap = ticker.info.get('marketCap')

# Get total debt
# yfinance provides 'totalDebt', but sometimes 'longTermDebt' + 'shortTermDebt' is more reliable
total_debt = ticker.info.get('totalDebt', ticker.info.get('longTermDebt', 0) + ticker.info.get('shortTermDebt', 0))


# Corrected print statements to use scale_factor for division within the f-string
print(f"Market Capitalization for {ticker_symbol}: ${market_cap / scale_factor:,.2f}")
print(f"Total Debt for {ticker_symbol}: ${total_debt / scale_factor:,.2f}")

# compute equity and debt weights
total_capitalization = market_cap + total_debt
w_E = market_cap / total_capitalization
w_D = total_debt / total_capitalization

print(f"Equity Weight for {ticker_symbol}: {w_E:,.2%}")
print(f"Debt Weight for {ticker_symbol}: {w_D:,.2%}")

Market Capitalization for MSFT: $3,769,404.75
Total Debt for MSFT: $120,375.00
Equity Weight for MSFT: 96.91%
Debt Weight for MSFT: 3.09%


## Cost of Equity

Really this is just beta because the other two are either given or looked up already.

In [13]:
# Download historical data for the firm and S&P 500
# get five years of monthly data
stock_data = yf.download(ticker_symbol, period='5y', interval='1mo')['Close']
index_data = yf.download(index_symbol, period='5y', interval='1mo')['Close']

# Calculate monthly returns for stock and S&P 500, and subtract off the risk free rate
stock_returns = stock_data.pct_change().dropna()
index_returns = index_data.pct_change().dropna()

# --- Regression (CAPM Beta) ---
X = sm.add_constant(index_returns)
model = sm.OLS((stock_returns), X).fit()

print(model.summary())

/tmp/ipython-input-3088552102.py:3: FutureWarning: YF.download() has changed argument auto_adjust default to True
  stock_data = yf.download(ticker_symbol, period='5y', interval='1mo')['Close']
[*********************100%***********************]  1 of 1 completed
/tmp/ipython-input-3088552102.py:4: FutureWarning: YF.download() has changed argument auto_adjust default to True
  index_data = yf.download(index_symbol, period='5y', interval='1mo')['Close']
[*********************100%***********************]  1 of 1 completed

                            OLS Regression Results                            
Dep. Variable:                   MSFT   R-squared:                       0.530
Model:                            OLS   Adj. R-squared:                  0.522
Method:                 Least Squares   F-statistic:                     64.30
Date:                Wed, 05 Nov 2025   Prob (F-statistic):           6.40e-11
Time:                        23:10:29   Log-Likelihood:                 100.94
No. Observations:                  59   AIC:                            -197.9
Df Residuals:                      57   BIC:                            -193.7
Df Model:                           1                                         
Covariance Type:            nonrobust                                         
                 coef    std err          t      P>|t|      [0.025      0.975]
------------------------------------------------------------------------------
const          0.0049      0.006      0.827      0.4

In [14]:
# Get the beta - it is the coefficient on the index return, not the constant
beta = model.params[1]

print(f"Beta for {ticker_symbol}: {beta:.2f}")

# Calculate the cost of equity using CAPM
cost_of_equity = (risk_free_rate + beta * emrp*100)/100

print(f"Cost of Equity for {ticker_symbol}: {cost_of_equity:.2%}")


Beta for MSFT: 1.07
Cost of Equity for MSFT: 9.39%


/tmp/ipython-input-3054460526.py:2: FutureWarning: Series.__getitem__ treating keys as positions is deprecated. In a future version, integer keys will always be treated as labels (consistent with DataFrame behavior). To access a value by position, use `ser.iloc[pos]`
  beta = model.params[1]


## Cost of Debt

Really, just the spread is all we need.

I have a table that I like to use from Aswath Damodaran.
[Here is the link.](https://people.stern.nyu.edu/adamodar/New_Home_Page/datafile/ratings.html)


I updated it with the following prompt in ChatGPT:

*I have the following list structure for credit spreads in python:*

```
Paste old code table here
```
*Please update it with this new data (copied from a web site):*
```
Paste unformatted table text from Damodaran's website here
```

And that seems to work






In [15]:
# updated as of Jan 2025:
credit_spreads = [
    {"GreaterThan": -100000, "LessThan": 0.199999, "Rating": "D2/D", "Spread": 19.00},
    {"GreaterThan": 0.2, "LessThan": 0.649999, "Rating": "C2/C", "Spread": 15.50},
    {"GreaterThan": 0.65, "LessThan": 0.799999, "Rating": "Ca2/CC", "Spread": 10.10},
    {"GreaterThan": 0.8, "LessThan": 1.249999, "Rating": "Caa/CCC", "Spread": 7.28},
    {"GreaterThan": 1.25, "LessThan": 1.499999, "Rating": "B3/B-", "Spread": 4.42},
    {"GreaterThan": 1.5, "LessThan": 1.749999, "Rating": "B2/B", "Spread": 3.00},
    {"GreaterThan": 1.75, "LessThan": 1.999999, "Rating": "B1/B+", "Spread": 2.61},
    {"GreaterThan": 2, "LessThan": 2.2499999, "Rating": "Ba2/BB", "Spread": 1.83},
    {"GreaterThan": 2.25, "LessThan": 2.49999, "Rating": "Ba1/BB+", "Spread": 1.55},
    {"GreaterThan": 2.5, "LessThan": 2.999999, "Rating": "Baa2/BBB", "Spread": 1.20},
    {"GreaterThan": 3, "LessThan": 4.249999, "Rating": "A3/A-", "Spread": .95},
    {"GreaterThan": 4.25, "LessThan": 5.499999, "Rating": "A2/A", "Spread": .85},
    {"GreaterThan": 5.5, "LessThan": 6.499999, "Rating": "A1/A+", "Spread": 0.77},
    {"GreaterThan": 6.5, "LessThan": 8.499999, "Rating": "Aa2/AA", "Spread": 0.60},
    {"GreaterThan": 8.5, "LessThan": 100000, "Rating": "Aaa/AAA", "Spread": 0.45}
]


Here we are going to write a function. Remember, we'll do this in two overall steps:

1. Write a script (not a function) that does what you want it to do and check and be sure that all the logic works.
2. Turn that script into a function with inputs and outputs, etc.

In [16]:
# write a function. It should take two inputs:
# 1. rating, a string variable that represents the firm's credit rating
# 2. credit_spreads, a list of dictionaries that has a lookup table from Damodaran's website.
# The table is in the above cell.
#
# The output should return a credit spread, which is a float data type.
# If no match is found, it should print a warning and return a NaN


# Example usage

def get_credit_spread(rating, credit_spreads):
    spread = next((d["Spread"] for d in credit_spreads if d["Rating"] == rating), np.nan)
    if np.isnan(spread):
        print(f"Warning: No credit spread found for rating: {rating}")
    return spread

# Example usage
firm_rating = "Aaa/AAA"
credit_spread = get_credit_spread(firm_rating, credit_spreads)

print(
    f"The spread for {ticker_symbol}'s rating ({firm_rating}) is: "
    f"{credit_spread/100:.2%}" if not np.isnan(credit_spread)
    else f"Could not find a credit spread for {ticker_symbol}'s rating ({firm_rating})"
)

The spread for MSFT's rating (Aaa/AAA) is: 0.45%


In [17]:
# compute cost of debt: risk free rate plus credit spread
# Ensure credit_spread is a decimal before adding to risk_free_rate
cost_of_debt = risk_free_rate/100 + (credit_spread / 100)

print(f"Cost of Debt for {ticker_symbol}: {cost_of_debt:.2%}")

Cost of Debt for MSFT: 4.51%


## Finally, compute the WACC

put it all together

In [18]:
# Calculate WACC
wacc = w_E * cost_of_equity + w_D * cost_of_debt * (1 - marg_tax_rate)

print(f"WACC for {ticker_symbol}: {wacc:.2%}")

WACC for MSFT: 9.20%


# Homework: What about Uncertainty?

Our estimate of beta has a range of uncertainty about it - a 95% confidence interval.

**HOMEWORK**

1. Go back to the OLS above and find the lower and upper CI for beta. Do this programmatically (i.e., find it in the `results` object)
2. Recompute a *lower* and *upper* WACC based on that *lower* and *upper* beta

If you are completely stuck, you should hard code the upper and Lower CI so you can turn this in, but there will be a deduction.

Expected output: A DataFrame, one column, three rows, with the Lower CI, the Estimate, and Upper CI version of the WACC, in that order.

In [19]:
# --- Step 1: Extract beta and its 95% confidence interval ---
beta = model.params.iloc[1]
ci_low, ci_high = model.conf_int().iloc[1]

In [20]:
cost_of_equity_low = (risk_free_rate + ci_low * emrp*100)/100
cost_of_equity_est = (risk_free_rate + beta * emrp*100)/100
cost_of_equity_high = (risk_free_rate + ci_high * emrp*100)/100

In [21]:
wacc_low = w_E * cost_of_equity_low + w_D * cost_of_debt * (1 - marg_tax_rate)
wacc_est = w_E * cost_of_equity_est + w_D * cost_of_debt * (1 - marg_tax_rate)
wacc_high = w_E * cost_of_equity_high + w_D * cost_of_debt * (1 - marg_tax_rate)

In [22]:
# Create and display WACC DataFrame as percentages
wacc_df = pd.DataFrame(
    {"WACC": [wacc_low, wacc_est, wacc_high]},
    index=["Lower CI", "Estimate", "Upper CI"]
)

with pd.option_context('display.float_format', '{:.2%}'.format):
    display(wacc_df)

,WACC
Lower CI,7.91%
Estimate,9.20%
Upper CI,10.49%
